In [107]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
from bs4 import BeautifulSoup
driver = webdriver.Chrome()

In [108]:
driver.get('https://aqicn.org/city/spain/catalunya/barcelona/')

In [109]:
historic_data_block = driver.find_element_by_id(
    'historic-aqidata-block'
)
actions = ActionChains(driver)
actions.move_to_element(historic_data_block)
actions.perform()
time.sleep(10)

In [110]:
# buscamos partícula pm10 por el xpath
pm10 = driver.find_element_by_xpath(
    '//*[@id="historic-aqidata-inner"]/div[2]/div[2]/center/ul/li[1]'
)
pm10.click()

In [111]:
historic_data = driver.find_element_by_class_name(
    'historical-yearly-data'
)
table = driver.find_element_by_class_name('squares')

In [112]:
soup = BeautifulSoup(driver.page_source)

In [113]:
table_data = soup.find(
    id='historic-aqidata-inner'
).find('table').find('tbody')

In [114]:
from datetime import datetime
data = []
for row in table_data.find_all('tr'):
    # cuando el dato está lleno tiene table-row
    # el year-divider es para el relleno de la tabla
    if row.get('class') == ['year-divider']:
        year = row.text
        print(year)
    if row['style'] == 'display: table-row;' and row.get(
        'class') != ['year-divider']:
        month = None
        for td in row.find_all('td'):
            if month is None:
                month = row.td.text
                print(month)
            if td.get('class') == ['squares']:
                # comenzamos con el día 1 cada mes
                day = 1
                for svg in td.find_all('svg'):
                    for rect in svg.find_all('text'):
                        timestamp = datetime.strptime(
                            f'{day}-{month}-{year}',
                            '%d-%b-%Y' 
                        )
                        day += 1
                        temp_data = {
                            'timestamp': timestamp,
                            'pm10': rect.text
                        }
                        data.append(temp_data)        

2020
Mar
Feb
Jan
2019
Dec
Nov
Oct
Sep
Aug
Jul
Jun
May


In [115]:
import pandas as pd
pm10_df = pd.DataFrame(data)
pm10_df.head()

,timestamp,pm10
0,2020-03-01,29
1,2020-03-02,46
2,2020-03-03,14
3,2020-03-04,8
4,2020-03-05,10


In [104]:
# write the data in a csv file
pm10_df.to_csv('../data/pm10.csv', index=False)